<a href="https://colab.research.google.com/github/DLuisMontufar3/1/blob/main/Si6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

\**06 Integración de Sistemas (ERP y SCM)**
Luis Montufar

In [20]:
import pandas as pd

In [21]:
filename = 'https://raw.githubusercontent.com/DLuisMontufar3/1/refs/heads/main/datos/inventario_ERPSCM.csv'

df = pd.read_csv(filename)


In [22]:
print(df.head())


    id                                   nombre   precio  stock_actual  \
0  100        Laptop Gamer Muestra (Bajo Stock)  1500.00             2   
1  101           Cable HDMI Básico (Stock Alto)    15.00           200   
2  102  Audífonos Noise Cancelling Lenovo Gamer   306.71            79   
3  103            Mouse Inalámbrico Xiaomi Slim    90.75            83   
4  104                  Lavadora Apple Business   585.53            17   

   punto_reorden  pronostico_venta  
0             10                25  
1             20                50  
2             26                88  
3             23                86  
4             26                23  


In [23]:
print(f"✅ Datos cargados: {len(df)} productos en el sistema ERP.")
# Mostramos los datos usando Pandas para que se vea bonita la tabla
display(pd.DataFrame(df))

✅ Datos cargados: 302 productos en el sistema ERP.


,id,nombre,precio,stock_actual,punto_reorden,pronostico_venta
0,100,Laptop Gamer Muestra (Bajo Stock),1500.00,2,10,25
1,101,Cable HDMI Básico (Stock Alto),15.00,200,20,50
2,102,Audífonos Noise Cancelling Lenovo Gamer,306.71,79,26,88
3,103,Mouse Inalámbrico Xiaomi Slim,90.75,83,23,86
4,104,Lavadora Apple Business,585.53,17,26,23
...,...,...,...,...,...,...
297,397,Smartphone Lenovo Pro,1325.42,79,22,80
298,398,Parlante Bluetooth Xiaomi Pro,139.87,67,30,71
299,399,Laptop HP Max,1927.56,71,26,68
300,400,Laptop LG Pro,1284.85,41,18,48


In [24]:
# 1. Reiniciamos la base de datos para asegurar una prueba limpia
ventas_totales_dia = 0.0

In [25]:

base_datos_erp = pd.read_csv(
    'https://raw.githubusercontent.com/DLuisMontufar3/1/refs/heads/main/datos/inventario_ERPSCM.csv'
)

# Convertimos a lista de diccionarios para poder iterar y modificar
base_datos_erp = base_datos_erp.to_dict('records')

In [26]:
def procesar_venta(id_producto, cantidad_deseada):
    global ventas_totales_dia
    producto_encontrado = False

    print(f"Se desea vender {cantidad_deseada} unidad(es) del producto {id_producto}...")

    for producto in base_datos_erp:
        if producto['id'] == id_producto:
            producto_encontrado = True

            # --- TAREA: Lógica ERP ---
            # Verifica si hay suficiente stock
            if producto['stock_actual'] >= cantidad_deseada:

                # Actualiza el stock
                producto['stock_actual'] -= cantidad_deseada # <--- COMPLETA AQUÍ

                # Calcula el ingreso (Precio * Cantidad)
                ingreso = producto['precio'] * cantidad_deseada # <--- COMPLETA AQUÍ
                ventas_totales_dia += ingreso

                print(f"   Transacción exitosa: {producto['nombre']}. Nuevo stock: {producto['stock_actual']}")
            else:
                print(f"   Error: Stock insuficiente para {producto['nombre']}.")

    if not producto_encontrado:
        print("   Error: Producto no encontrado.")

print("Módulo ERP cargado.")

Módulo ERP cargado.


In [27]:
def ejecutar_modelo_push():
    print("\nEJECUTANDO MODELO SCM PUSH (MAKE-TO-STOCK)")
    print("Regla: Comprar anticipadamente si Stock Actual < Pronóstico.")

    compras_realizadas = 0

    for producto in base_datos_erp:
        stock = producto['stock_actual']
        pronostico = producto['pronostico_venta']

        # --- TAREA: Lógica Push ---
        # Si el stock actual es menor al pronóstico, debemos comprar la diferencia
        if stock < pronostico:

            # Calcula cuánto falta para alcanzar el pronóstico
            cantidad_a_comprar = pronostico - stock  # <--- COMPLETA AQUÍ

            print(f"   PUSH: {producto['nombre']} (Stock: {stock} | Pronóstico: {pronostico})")
            print(f"   Generando orden de compra por {cantidad_a_comprar} unidades.")

            # Simulamos llegada de mercadería
            producto['stock_actual'] += cantidad_a_comprar
            compras_realizadas += 1

    if compras_realizadas == 0:
        print("   PUSH Reporte: Inventario suficiente para los pronósticos.")

print("Módulo SCM (Push) cargado.")

Módulo SCM (Push) cargado.


In [28]:
def ejecutar_modelo_pull(id_producto, cantidad_cliente):
    print(f"\n🚚 EJECUTANDO MODELO PULL (MAKE-TO-ORDER)")
    print(f"Cliente solicita {cantidad_cliente} unidades del producto {id_producto}...")

    for producto in base_datos_erp:
        if producto['id'] == id_producto:
            stock = producto['stock_actual']

            # --- TAREA 3: Lógica Pull ---
            if stock >= cantidad_cliente:
                print(f"   ✅ Stock suficiente ({stock}). Se despacha del inventario existente.")
                producto['stock_actual'] -= cantidad_cliente
            else:
                # Si no hay stock, el modelo Pull dispara la compra JUSTO AHORA
                faltante = cantidad_cliente - stock # <--- COMPLETA AQUÍ

                print(f"   Stock insuficiente. Faltan {faltante} unidades.")
                print(f"   -> PULL ACTIVADO: Comprando {faltante} unidades urgentemente...")

                print(f"   ✅ Pedido entregado al cliente (Just-in-Time).")
                # En Pull puro, el stock queda en 0 tras la entrega inmediata (si no se pidió extra)
                producto['stock_actual'] = 0

print("Módulo SCM (Pull) cargado.")

Módulo SCM (Pull) cargado.


In [29]:
# 1. Reiniciamos la base de datos para asegurar una prueba limpia
ventas_totales_dia = 0.0

print("--- 1. PROBANDO ERP (VENTAS) ---")
procesar_venta(101, 1)  # ¿Funciona o falla?
procesar_venta(103, 10) # ¿Funciona o falla?

# Se debe crear un caso en el que la venta funcione,
# y otro caso en el que la venta falle

print(f"\nDinero en Caja (Finanzas): ${ventas_totales_dia}")

print("\n--- 2. PROBANDO SCM (PUSH) ---")
# Esto debería reabastecer un producto que no se haya podido vender
ejecutar_modelo_push()

print("\n--- 3. PROBANDO SCM (PULL) ---")
# Se debe pedir un producto en una cantidad superior al stock
# El sistema Pull debería comprar los faltantes automáticamente
ejecutar_modelo_pull(105, 200)

print("\n📊 ESTADO FINAL DEL INVENTARIO (ERP INTEGRADO)")
# Convertimos de nuevo a DataFrame para ver la tabla bonita en Jupyter
display(pd.DataFrame(base_datos_erp))

--- 1. PROBANDO ERP (VENTAS) ---
Se desea vender 1 unidad(es) del producto 101...
   Transacción exitosa: Cable HDMI Básico (Stock Alto). Nuevo stock: 199
Se desea vender 10 unidad(es) del producto 103...
   Transacción exitosa: Mouse Inalámbrico Xiaomi Slim. Nuevo stock: 73

Dinero en Caja (Finanzas): $922.5

--- 2. PROBANDO SCM (PUSH) ---

EJECUTANDO MODELO SCM PUSH (MAKE-TO-STOCK)
Regla: Comprar anticipadamente si Stock Actual < Pronóstico.
   PUSH: Laptop Gamer Muestra (Bajo Stock) (Stock: 2 | Pronóstico: 25)
   Generando orden de compra por 23 unidades.
   PUSH: Audífonos Noise Cancelling Lenovo Gamer (Stock: 79 | Pronóstico: 88)
   Generando orden de compra por 9 unidades.
   PUSH: Mouse Inalámbrico Xiaomi Slim (Stock: 73 | Pronóstico: 86)
   Generando orden de compra por 13 unidades.
   PUSH: Lavadora Apple Business (Stock: 17 | Pronóstico: 23)
   Generando orden de compra por 6 unidades.
   PUSH: Parlante Bluetooth Canon RGB (Stock: 2 | Pronóstico: 23)
   Generando orden de com

,id,nombre,precio,stock_actual,punto_reorden,pronostico_venta
0,100,Laptop Gamer Muestra (Bajo Stock),1500.00,25,10,25
1,101,Cable HDMI Básico (Stock Alto),15.00,199,20,50
2,102,Audífonos Noise Cancelling Lenovo Gamer,306.71,88,26,88
3,103,Mouse Inalámbrico Xiaomi Slim,90.75,86,23,86
4,104,Lavadora Apple Business,585.53,23,26,23
...,...,...,...,...,...,...
297,397,Smartphone Lenovo Pro,1325.42,80,22,80
298,398,Parlante Bluetooth Xiaomi Pro,139.87,71,30,71
299,399,Laptop HP Max,1927.56,71,26,68
300,400,Laptop LG Pro,1284.85,48,18,48


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['id'].plot(kind='hist', bins=20, title='id')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['precio'].plot(kind='hist', bins=20, title='precio')
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['pronostico_venta']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_11.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('pronostico_venta')

from matplotlib import pyplot as plt
_df_12['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_13['id'].plot(kind='line', figsize=(8, 4), title='id')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_14['precio'].plot(kind='line', figsize=(8, 4), title='precio')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_15['stock_actual'].plot(kind='line', figsize=(8, 4), title='stock_actual')
plt.gca().spines[['top', 'right']].set_visible(False)

In [30]:
ejecutar_modelo_pull(105, 200)


🚚 EJECUTANDO MODELO PULL (MAKE-TO-ORDER)
Cliente solicita 200 unidades del producto 105...
   Stock insuficiente. Faltan 200 unidades.
   -> PULL ACTIVADO: Comprando 200 unidades urgentemente...
   ✅ Pedido entregado al cliente (Just-in-Time).


In [31]:
ejecutar_modelo_pull(100, 30)


🚚 EJECUTANDO MODELO PULL (MAKE-TO-ORDER)
Cliente solicita 30 unidades del producto 100...
   Stock insuficiente. Faltan 5 unidades.
   -> PULL ACTIVADO: Comprando 5 unidades urgentemente...
   ✅ Pedido entregado al cliente (Just-in-Time).


In [32]:
ejecutar_modelo_pull(101, 300)


🚚 EJECUTANDO MODELO PULL (MAKE-TO-ORDER)
Cliente solicita 300 unidades del producto 101...
   Stock insuficiente. Faltan 101 unidades.
   -> PULL ACTIVADO: Comprando 101 unidades urgentemente...
   ✅ Pedido entregado al cliente (Just-in-Time).
